In [72]:
# 1. 모듈 연결하기
import requests
import pandas as pd
import json

ModuleNotFoundError: No module named 'tqdm'

In [33]:
# 2. 기본정보 설정
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params ={'serviceKey' : 'V2soxVa/BMDzYDIk9IbGZTgcZeSQ6GIdaZnsBoez7GPAUwIQ/6MT/jyI5mtbTgXy24zNvlE9d5/EF0Y0YoKoNQ==', 
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'dataType' : 'JSON', 
         'dataCd' : 'ASOS', 
         'dateCd' : 'DAY', 
         'startDt' : '20100101', 
         'endDt' : '20100806', 
         'stnIds' : '108' }

In [34]:
def loc_code_sch(loc_name):
    loc_code = pd.read_csv('./기상청위치코드.csv', encoding='utf-8')
    loc_code=loc_code[loc_code['지점명']==loc_name]['지점'].values[0]
    return str(loc_code)

In [35]:
# 사용자 검색옵션 입력 및 지역 코드 확인 
startDT = input('시작일자를 입력하세요(yyyymmdd): ')
endDT= input('종료일자를 입력하세요(yyyymmdd): ')
location= input('지역을 입력하세요: ')
loc_code=loc_code_sch(location)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [36]:
#4. 사용자가 입력받은 조건으로 기상청에 데이터 요청하기 
params['startDt'] = startDT
params['endDt'] = endDT
params['stnIds'] = loc_code

params

{'serviceKey': 'V2soxVa/BMDzYDIk9IbGZTgcZeSQ6GIdaZnsBoez7GPAUwIQ/6MT/jyI5mtbTgXy24zNvlE9d5/EF0Y0YoKoNQ==',
 'pageNo': '1',
 'numOfRows': '10',
 'dataType': 'JSON',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '',
 'endDt': '20230101',
 'stnIds': '108'}

In [40]:
# 5. 기상청 조회 일자별 데이터 가져오기
response = requests.get(url, params=params)

weather = json.loads(response.content)
weather

{'response': {'header': {'resultCode': '02', 'resultMsg': 'DB_ERROR'}}}

In [41]:
#6. 전체 조회데이터 건수를 이용한 데이터 한페이지 출력 
params['numOfRows'] = weather['response']['body']['totalCount']
response = requests.get(url, params=params)

weather = json.loads(response.content)
weather['response']['body']['items']['item'][0]

KeyError: 'body'

In [27]:
#7. 분석에 필요한 데이터 추출 
weather_df = pd.DataFrame(weather['response']['body']['items']['item'])
weather_df.head(3)

NameError: name 'weather' is not defined

In [29]:
#8 문서에 필요한 데이터 추출 
wt_df = weather_df[['tm','stnNm','avgTa','minTa','maxTa','sumRn']].copy()
wt_df.info()

NameError: name 'weather_df' is not defined

In [42]:
#지점명을 이용한 지점코드 찾기 
def loc_code_sch(loc_name):
    loc_code = pd.read_csv('./기상청위치코드.csv', encoding ='utf-8')
    loc_code = loc_code[loc_code['지점명'] == loc_name]['지점'].values[0]
    return str(loc_code)

In [50]:
### 기상청 api를 이용한 전체데이터(1904~)가져오기

def weather_api(loc_name, startDt, endDt, stnIds, numOfRows=10):
    
    #지역명으로 지점코드 조회 
    stnIds = loc_code_sch(loc_name)
    
    # 기상청 api 호출을 위한 기본정보 설정
    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
    params ={'serviceKey' : 'V2soxVa/BMDzYDIk9IbGZTgcZeSQ6GIdaZnsBoez7GPAUwIQ/6MT/jyI5mtbTgXy24zNvlE9d5/EF0Y0YoKoNQ==', 
         'pageNo' : '1', 
         'numOfRows' : numOfRows, 
         'dataType' : 'JSON', 
         'dataCd' : 'ASOS', 
         'dateCd' : 'DAY', 
         'startDt' : startDt, 
         'endDt' : endDt, 
         'stnIds' : stnIds }
    
    #기상청 데이터 조회 
    response = requests.get(url, params=params) #api 호출 
    if response.status_code != 200:
         exit('API 호출 에러 발생')
         
    weather_res = json.loads(response.content)
    return weather_res # 결과반환 

In [51]:
# 최초 데이터 요청 
startDT = input('시작일자를 입력하세요(yyyymmdd): ')
endDT= input('종료일자를 입력하세요(yyyymmdd): ')
loc_name= input('지역을 입력하세요: ')

weather_res = weather_api(loc_name, startDT, endDT)
numOfRows = weather_res['response']['body']['totalCount']

weather_res = weather_api(loc_name, startDT, endDT, numOfRows)
weather_df = pd.DataFrame(weather_res['response']['body']['items']['item'])
weather_df.info()

TypeError: weather_api() missing 1 required positional argument: 'stnIds'

In [58]:
#1960.01.01 ~ 오늘 -1일 까지의 기상청 일자별 데이터를 가져온다.
pd.date_range(start='1960-01-01', end='2022-12-31', freq='12M') #freq D-일별 M-달별 

DatetimeIndex(['1960-01-31', '1961-01-31', '1962-01-31', '1963-01-31',
               '1964-01-31', '1965-01-31', '1966-01-31', '1967-01-31',
               '1968-01-31', '1969-01-31', '1970-01-31', '1971-01-31',
               '1972-01-31', '1973-01-31', '1974-01-31', '1975-01-31',
               '1976-01-31', '1977-01-31', '1978-01-31', '1979-01-31',
               '1980-01-31', '1981-01-31', '1982-01-31', '1983-01-31',
               '1984-01-31', '1985-01-31', '1986-01-31', '1987-01-31',
               '1988-01-31', '1989-01-31', '1990-01-31', '1991-01-31',
               '1992-01-31', '1993-01-31', '1994-01-31', '1995-01-31',
               '1996-01-31', '1997-01-31', '1998-01-31', '1999-01-31',
               '2000-01-31', '2001-01-31', '2002-01-31', '2003-01-31',
               '2004-01-31', '2005-01-31', '2006-01-31', '2007-01-31',
               '2008-01-31', '2009-01-31', '2010-01-31', '2011-01-31',
               '2012-01-31', '2013-01-31', '2014-01-31', '2015-01-31',
      

In [66]:
from datetime import date
stDY= 2000
endDY=2020
for i in range(stDY, endDY+1):
    print(date(i, 1, 1).strftime('%Y%m%d')) # Y: yyyy, m: mm, d: dd , str from time 시간을 문자로 바꿔라
    print(date(i, 12, 31).strftime('%Y-%m-%d'))

20000101
20010101
20020101
20030101
20040101
20050101
20060101
20070101
20080101
20090101
20100101
20110101
20120101
20130101
20140101
20150101
20160101
20170101
20180101
20190101
20200101


In [71]:
from datetime import date, datetime, timedelta   # 날짜/시간 관련 모듈
  # 진행바 모듈
import tqdm

startDT = input('시작연도(yyyy): ')
endDT= input('종료년도(yyyy): ')
loc_name= input('지역을 입력하세요: ')

weather_df=pd.DataFrame()   # 최초 데이터프레임 생성

for i in tqdm.tqdm(range(int(startDT), int(endDT)+1)):
    
    startDT = date(i, 1, 1).strftime("%Y%m%d")
    endDT = date(i, 12, 31).strftime("%Y%m%d")
    today = date.today().strftime("%Y%m%d")
    
    if int(endDT) >= int(today):
        endDT = int(date.today().strftime("%Y%m%d")) - 1   # 20230807-1 = 20230806
        endDT = str(endDT)
    
    weather_res = weather_api(loc_name, startDT, endDT)   # 함수 호출(최초)
    numOfRows = weather_res['response']['body']['totalCount']

    weather_res = weather_api(loc_name, startDT, endDT, numOfRows)   # 함수 호출(전체 조회 데이터)
    df1 = pd.DataFrame(weather_res['response']['body']['items']['item'])
    weather_df = pd.concat([weather_df, df1], ignore_index=True)   # 데이터프레임 병합
    
weather_df.info()



ModuleNotFoundError: No module named 'tqdm'